# Trabajo final: Diplomatura en Deep Learning - Año 2020
## Alumno: Germán Dima

# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [ ]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

In [ ]:
# %tensorflow_version 1.x   # en Colab

In [ ]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

# por si tira "tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead"
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [ ]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida
directorio = "D:/Cosas Aca/Tp Final"

base_image_path = Path(directorio + "/content/775px-Neckarfront_Tübingen_Mai_2017.jpg")
style_reference_image_path = Path(directorio + "/content/La_noche_estrellada1.jpg")
result_prefix = Path(directorio + "/content/output")
iterations = 100

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:

*total_variation_weight* = Peso de la *Loss* de suavizado, a la *Loss* global. Regula cuánta importancia se le da a que la imagen final no presente regiones ruidosas de definición (principalmente en los bordes).

*style_weight* = Peso de la *Loss* de *Style*, a la *Loss* global (ec. 7 del artículo). Regula cuánta importancia tendrá el estilo de entrada sobre la imagen de salida.

*content_weight* = Peso de la *Loss* de *Content*, a la *Loss* global (ec. 7 del artículo). Regula cuánta importancia tendrá el contenido de entrada sobre la imagen de salida.


In [ ]:
total_variation_weight = 0.1
style_weight = 10
content_weight = 1
# Recomiendan en el artíuclo content_weight / style_weight ~ 10^-3 , 10^-4 (en el caso total_variation_weight  = 0)

In [ ]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta:

La siguiente función (1) carga una imagen y la re-escalea a dimensión: (img_nrows x img_ncols), (2) la convierte en un array numérico de dimensión: (img_nrows,img_ncols,3 -que es el número de canales-), (3) para el manejo en batch, agrega una dimensión extra en la primera componente: (1,img_nrows,img_ncols,3), (4) para un correcto funcionamiento de la VGG19, el array es convertido de RGB a BGR, para luego centrar cada canal (sin normalizar) respecto del dataset ImageNet (con el que fue entrenado la red).

In [ ]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta:

La siguiente función puede pensarse como la inversa de la función "preprocess_image", ya que (1) convierte una imagen al formato (alto,ancho,canales) -ignorando la componente del batch size-, (2) descentra cada canal respecto de los valores medios del dataset ImageNet (103.939, 116.779, 123.68) y (3) vuelve a reordenar para pasar del formato BGR a RGB. El comando "clip", en el rango 0 a 255, garantiza que sea un color válido.

In [ ]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [ ]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [ ]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [ ]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?
- ¿Por qué se permutan las dimensiones de x?

La matriz de Gram almacena la información de las correlaciones entre las distintas respuestas de los filtros. Al ser generada como el producto interno entre filtros (aplanados), esta matriz es usada para captar la esencia de estilos (altos valores en sus elementos implicarían que los filtros que lo componen se han activado en esa posición espacial). Matemáticamente se busca minimizar la distancia cuadrática media entre la matriz de Gram de la imagen orginal (de estilo) y la generada por el modelo.

Según su definición en el artículo (ec. 3), el elemento (i,j) de la matriz se computa como el producto escalar sobre los filtros $F_{i,j}$ ahora pensados como vectores. Esto nos devuelve una matriz cuadrada cuya dimensiones corresponden al producto de las dimensiones de alto y ancho de los mapas de _features_.

Yendo a la definición de la siguiente celda, si "x" es de dimensión (25,32,512) -alto,ancho,cantidad de filtros en esa capa-, el comando "permute" lo transformará en dimensión (512,25,32) para luego pasara a dimensión (512,800) mediante el "flatten" (aplana "alto" y "ancho"). Por lo tanto:
$\overline{features}\in {{\mathbb{R}}^{\text{512}\times \text{800}}}* {{\overline{features}}^{T}}\in {{\mathbb{R}}^{\text{800}\times \text{512}}}=gram\in {{\mathbb{R}}^{\text{512}\times 512}}$

In [ ]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:

La función "style_loss" da una noción de similitud de estilos entre la imagen generada y la original (de estilo). Computa la contribución de una capa en particular a la *Loss* general (ec. 4) mediante la distancia cuadrática media entre las matrices de Gram (de la imagen original y la generada).

La función "content_loss" responde al contenido entre la imagen generada frente a la original (de contenido). Se calcula mediante el error cuadrático (ec. 1) entre el array de una capa y la imagen original.

La función "total_variation_loss" es un suavizado espacial, actuándo como regularizador de la imagen. Computa la suma de las diferencias absolutas de los valores de píxeles vecinos. Su minimización conlleva a un filtro de picos abruptos de intensidad de píxeles.

In [ ]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [ ]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [ ]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [ ]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [ ]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:

En el artículo original los autores utilizan el método de BFGS para la minimización de la *Loss*. Este algoritmo determina en qué dirección desplazarse y la distancia a avanzar, mediante el cálculo de la Hessiano. El método requiere que se pase el valor de la *Loss* y el valor del gradiente como funciones por separado. Adicionalmente, el optimizador opera sobre vectores planos (en vez de arrays de dimensión tres). La función de la celda siguiente apunta en resolver estos dos asuntos. 

En este código se utiliza una variante del optimizador: L-BFGS-B (que viene con SciPy, y se llama mediante "fmin_l_bfgs_b") el cual permite agregar condiciones de borde constantes a los parámetros (${{\min }_{i}}\le {{\theta }_{i}}\le {{\max }_{i}}$) además de un uso más restringido de memoria.

Como dice el comentario de la celda, no sería eficiente calcular el valor de la *Loss* y el valor de los gradientes de forma independiente. Para evitar esto, se crea una clase de llamada *Evaluator* la cual calcula estos valores a la vez, devolviendo el valor de las *Loss* cuando se la llame la primera vez y almacenará en caché los gradientes para las siguientes llamadas.

El post del blog https://blog.slavv.com/picking-an-optimizer-for-style-transfer-86e7b8cba84b muestra una serie de experimentos en donde prueban diferentes alternativas de optimizadores para este tipo de procesamiento de imágenes. Si bien Adagrad, Adam y L-BFGS muestran una performance superior, este último presenta una significativa mejoría. Una de las razones podría atribuirse a que no existe ningún proceso estocástico en el cálculo: el optimizador siempre recibe la misma imagen (determinística).

In [ ]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [ ]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [ ]:
9hb 8i0evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(/u83):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

In [ ]:
# Para colab
if(False):
    from google.colab import drive, files
    drive.mount('/content/drive')
    
    # Zippeo las imágenes y las descargo    
    !zip -r "/content/test" "/content/output"
    files.download("test.zip") 
    
    # Nombre
    file_name = file_name = str("generadas_" + str(total_variation_weight) + "_" + str(style_weight) + "_" + str(content_weight) + ".zip")

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta:

Para poder estudiar la importancia de los pesos, se optó por utilizar una imagen de referencia (generada con parámetros ${{\left( \text{total }\!\!\_\!\!\text{ variation},\text{style},\text{content} \right)}_{\text{weight}}}\text{ = }\left( \text{0}\text{.1}\text{,10}\text{,1} \right)$ y compaginar todas las imágenes en una sola para una mejor comparación de las mismas. Adicionalmente, para seguir lo pedido en el enunciado, se han adjuntando las imágenes por separado.

Al aumentar el valor del peso de la *Loss* de suavizado, se observa un aumento de importancia a la continuidad de trazos de aquella *Loss* dominante. En la primera fila de la siguiente figura se han generado imágenes cuyo peso de la *Loss* de *syle* es diez veces mayor al del *content*, para luego ir incrementando el valor del peso del suavizado, sin modificar los restantes hiperparámetros. Se aprecia que estos cambios enfatizan los colores, previniendo saltos de tonalidades dados por la contribución del estilo. Por el contrario, si la relación entre estilo y contenido se invierte (segunda fila), el suavizado ataca a la rugosidad de la imagen (ver los techos de las casas). Cuando esta *Loss* adquiere un peso significativamente mayor a las anteriores, esta homogeneización provoca un efecto de desenfoque. Por otro lado, no se han encontrado diferencias a ojo desnudo para valores de esta *Loss* iguales a 0.005 y 0.

<img src="./figuras/test_suavidad.png">

In [ ]:
load_img("./figuras/test_suavidad.png")

In [ ]:
load_img("./figuras/generadas_0.1_10_1.png")   # total_variation = 0.1 / style = 10 / content = 1

In [ ]:
load_img("./figuras/generadas_10_10_1.png")    # total_variation = 10 / style = 10 / content = 1

In [ ]:
load_img("./figuras/generadas_100_10_1.png")    # total_variation = 100 / style = 10 / content = 1

In [ ]:
load_img("./figuras/generadas_0.1_1_10.png")    # total_variation = 0.1 / style = 1 / content = 10

In [ ]:
load_img("./figuras/generadas_10_1_10.png")    # total_variation = 10 / style = 1 / content = 10

In [ ]:
load_img("./figuras/generadas_100_1_10.png")    # total_variation = 100 / style = 1 / content = 10

Como fue mencionado anteriormente, los hiperparámetros *style_weight* y *content_weight* actúan como reguladores del estilo y contornos en la imagen generada. Para una mejor apreciación de sus roles se han generado imágenes con valores extremos. En la primera fila de la siguiente figura se aprecia como, al aumentar el hiperparámetro *style_weight*, no sólo los colores empiezan a virar hacia la imagen de referencia, sino también ciertas regiones empiezan a deformarse (ver el cielo a la altura de los tejados y la pared en contacto con el agua). El mismo fenómeno puede apreciarse al exagerarse el hiperparámetro *content_weight* (segunda fila), en donde el cielo y las casas tienden a parecerse al de la figura de referencia, quedando apenas leves irregularidades, del aporte del cuadro.

<img src="./figuras/test_estilo_bordes.png">

In [ ]:
load_img("./figuras/test_estilo_bordes.png")

In [ ]:
load_img("./figuras/generadas_0.1_1_1.png")  # total_variation = 0.1 / style = 1 / content = 1

In [ ]:
load_img("./figuras/generadas_0.1_10_1.png")    # total_variation = 0.1 / style = 10 / content = 1 (referencia)

In [ ]:
load_img("./figuras/generadas_0.1_1000_1.png")    # total_variation = 0.1 / style = 1000 / content = 1

In [ ]:
load_img("./figuras/generadas_0.1_10_0.1.png")  # total_variation = 0.1 / style = 10 / content = 0.1

In [ ]:
load_img("./figuras/generadas_0.1_10_1000.png")    # total_variation = 0.1 / style = 10 / content = 1000

# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta:

Del mismo modo que en la respuesta anterior, se adjunta una imagen global (con las imágenes originales de bordes y estilos, y la generada por el modelo) y luego cada imagen por separado. En los comentarios se encuentran los pesos utilizados en el formato ${{\left( \text{total }\!\!\_\!\!\text{ variation},\text{style},\text{content} \right)}_{\text{weight}}}$.

# <img src="./figuras/personal_01.png">

In [ ]:
load_img("./figuras/personal_01.png")   # Pesos de las Loss: (0.5 , 3 , 1)

In [ ]:
load_img("./figuras/personal_01_content.jpg")   # Imagen de "content"

In [ ]:
load_img("./figuras/personal_01_style.jpg")   # Imagen de "syle" (rotada para mejores resultados)

In [ ]:
load_img("./figuras/personal_01_final.png")   # Imagen generada

# <img src="./figuras/personal_02.png">

In [ ]:
load_img("./figuras/personal_02.png")   # Pesos de las Loss: (0.1 , 3 , 1)

In [ ]:
load_img("./figuras/personal_02_content.jpg")   # Imagen de "content"

In [ ]:
load_img("./figuras/personal_02_style.jpg")   # Imagen de "syle"

In [ ]:
load_img("./figuras/personal_02_final.png")   # Imagen generada

#  <img src="./figuras/personal_03.png">

In [ ]:
load_img("./figuras/personal_03.png")   # Pesos de las Loss: (0.1 , 3 , 1)

In [ ]:
load_img("./figuras/personal_03_content.jpg")   # Imagen de "content"

In [ ]:
load_img("./figuras/personal_03_style.jpg")   # Imagen de "syle"

In [ ]:
load_img("./figuras/personal_03_final.png")   # Imagen generada

# <img src="./figuras/personal_04.png">

In [ ]:
load_img("./figuras/personal_04.png") 
# Pesos de las Loss: (0.5 , 5 , 1), aumento en la loss de suavizado para resaltar más el fondo.

In [ ]:
load_img("./figuras/personal_04_content.jpg")   # Imagen de "content"

In [ ]:
load_img("./figuras/personal_04_style.jpg")   # Imagen de "syle"

In [ ]:
load_img("./figuras/personal_04_final.png")   # Imagen generada